In [1]:
# selenium
from selenium.webdriver.common.by import By
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#sys libraries
from time import sleep
import requests
import random
import csv
import io
import os
import sys
import types
import getpass
import string
import re
from bs4 import BeautifulSoup

# Import data viz libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# nltk and wordcloud
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

%matplotlib inline

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows' , 100)

In [2]:
# getting from user input
# try:
#     username = input('Enter your Username ')
#     password = getpass.getpass('Enter your password ')
#     desired_job = input('Searched job query: ')
#     desired_location = input('Location: ')
# except Exception as error: 
#     print('ERROR', error) 
    
    
username = 'behdad.k7@gmail.com'
password = '1988behrooz10973'
desired_keyword = 'data scientist in united kingdom'
desired_location = 'United Kingdom'

csv_file = str.strip(desired_keyword)
file_name = csv_file + "_" + str(desired_location) +'.csv'

In [7]:
driver = webdriver.Firefox(executable_path= "geckodriver.exe")

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.cache.disk.enable", False)
profile.set_preference("browser.cache.memory.enable", False)
profile.set_preference("browser.cache.offline.enable", False)
profile.set_preference("network.http.use-cache", False)
profile.update_preferences()

driver.get(url='https://www.linkedin.com/')
sleep(2)

# Find the sign in botthon
sign_in = driver.find_element(by='link text', value='Sign in')
sign_in.click()
sleep(5)

# We find the input section and enter our user information.
email_entry = driver.find_element(by='css selector', value='#username')
password_entry = driver.find_element(by='css selector', value='#password')

# For every letter in our email, we will set a random time between keystrokes
for letter in username:
    sleep(random.uniform(.1, .4))
    email_entry.send_keys(letter)

# We expect a random time between 0.1 and 0.4 sec for each letter of our password.
for letter in password:
    sleep(random.uniform(.1, .4))
    password_entry.send_keys(letter)

# We press enter
password_entry.send_keys(Keys.RETURN)
sleep(10)

# click on typehead search bar
search_bar = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search']")))
search_bar.click()

# Enter the job search term and press enter
for letter in desired_keyword:
    sleep(random.uniform(.1, .4))
    search_bar.send_keys(letter)
search_bar.send_keys(Keys.RETURN)
sleep(3)


# Find and Click on just people button
just_people_btn = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//button[@aria-label='View only People results']")))
just_people_btn.click()
sleep(2)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
current_url = 'url_placeholder' # this is a placeholder for the URL check

# Create empty dataframe
df = pd.DataFrame(columns = ['name', 'job_title', 'current_company', 'university',
                             'degree','location', 'about', 'skills','profile_url'])

# Go through pages and download data
current_url = driver.current_url
while True:
    # Check to see if url is the 100th page in search
    if current_url.find('page=100') != -1:
        break
    # Check to see if this url has been scraped before; break loop if it has
    previous_url = current_url
    current_url = driver.current_url
    if current_url == previous_url:
        break

    driver.get(present_url)
    sleep(10)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)
    
    links = driver.find_elements_by_xpath('//div[contains(@class, "search-result__info")]//a[@href]')
    profiles_link = set()
    for link in links:
        link_add=link.get_attribute('href')
        print(link_add)
        if link_add.startswith('https://www.linkedin.com/in/'):
            profiles_link.add(link_add)
            
    for profile in profiles_link:
        try:
            driver.get(profile)
            sleep(5)
        except:
            pass
        
        # -------------Scrap here----------------
            
        print('---------------------------')
        print('scraping ...' , profile , '\n')
        sleep(3)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        sleep(.75)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        sleep(.75)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        sleep(.75)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        sleep(.75)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        sleep(1)
        
        page = BeautifulSoup(driver.page_source, 'lxml')
        
        ''' 1)Name'''
        try:
            name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[contains(@class, "inline t-24 t-black t-normal break-words")]')))
            name = name.text
            if name:
                name = name.strip()
        except:
            name = np.nan
                
        print('name:\n',name , '\n')
        sleep(0.5)
        
        ''' 2)job_title'''
        try:
            job_title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//h2[contains(@class, "mt1 t-18 t-black t-normal")]')))
            job_title = job_title.text
            if job_title:
                job_title = job_title.strip()
        except:
            job_title = np.nan
                
        print('job_title:\n',job_title , '\n')
        sleep(0.5)
        
        ''' 3)current_company'''
        try:
            current_company = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--experience-list")]/li[1]')))
            current_company = current_company.text
            if current_company:
                current_company = current_company.strip()
        except:
            current_company = np.nan
                
        print('current_company:\n',current_company , '\n')
        sleep(0.5)
        
        ''' 4)university'''
        try:
            university = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--experience-list")]/li[2]')))
            if university =='':
                degree = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((
                        By.XPATH,'//section[contains(@class, "pv-profile-section education-section ember-view")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[1]//div[contains(@class,"pv-entity__degree-info")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//h3[contains(@class,"pv-entity__school-name t-16 t-black t-bold")]')))
            
            university = university.text
            if university:
                university = university.strip()
        except:
            university = np.nan
                

        print('university:\n',university , '\n')
        sleep(0.5)
        
        
        ''' 5)degree'''
        try:
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//section[contains(@class, "pv-profile-section education-section ember-view")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[1]//div[contains(@class,"pv-entity__degree-info")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//span[contains(@class,"pv-entity__comma-item")]')))
            degree = degree.text
            if degree=='':
                degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//p[contains(@class,"pv-entity__degree-name")]')))
                degree = degree.text
            
            if degree:
                degree = degree.strip()
        except:
            degree = np.nan
                
        print('degree:\n',degree , '\n')
        sleep(0.5)
        
        ''' 6)location'''
        try:
            location = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--list pv-top-card--list-bullet mt1")]/li[1]')))
            location = location.text
            if location:
                location = location.strip()
        except:
            location = np.nan
        
        print('location:\n',location , '\n')
        sleep(0.5)
        
        ''' 7)about'''
        try:
            about = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//section[contains(@class, "artdeco-container-card pv-profile-section pv-about-section ember-view")]//a[contains(@class,"lt-line-clamp__more")]'))).click()
            about = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//p[contains(@class,"pv-about__summary-text mt4 t-14 ember-view")]')))      
            about = about.text
            if about:
                about = about.strip()
        except:
            about = np.nan
             
        print('about:\n',about , '\n')
        sleep(0.5)
        
        ''' 8)skills'''
        
        try:      
            print("skills search ... ")
            skills = page.find_all('span', class_ = "pv-skill-category-entity__name-text")
            
            #Put scraped data into a ski_df
            arraylen3 = len(page.find_all('span', class_ = "pv-skill-category-entity__name-text"))
        
            profile = pro
            skills = list(map(lambda x: x.text.strip(), skills))[0:arraylen3]
        except:
            skills = np.nan
                
        print('skills:  ',skills , '\n')
        sleep(0.5)


        
        temp = pd.DataFrame({'name': name, 'job_title': job_title, 'current_company': current_company,
                             'university': university, 'degree': degree, 'location':location, 
                             'about':about,'skills': skills, 'profile_url': profile})

        # Filter out members who do not provide information
        temp = temp[temp['name'] != 'LinkedIn Member']

        # Append new data to df
        df = df.append(temp)

    # Find next button and hit next
    nextt = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//button[contains(@class,'artdeco-pagination__button artdeco-pagination__button--next artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary')]")))
    sleep(2)
    nextt.click()
    sleep(2)

# Reset dataframe index
df.reset_index()

# Export results
df.to_csv("UK_data_scientist.csv", index = False)

# Close Selenium
driver.quit()
